# **Development**

# Defining the Intentions of a Chatbot

In [6]:
data = {"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "createaccount",
    "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
    "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    }
]
}

with open('intents.json', 'w') as outfile:
    json.dump(data, outfile)

# Data preparation

In [7]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [8]:
with open('intents.json') as file:
    data = json.load(file)


training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [5]:
training_sentences

['Hi',
 'Hey',
 'Is anyone there?',
 'Hello',
 'Hay',
 'Bye',
 'See you later',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Thanks for the help',
 'Who are you?',
 'What are you?',
 'Who you are?',
 'what is your name',
 'what should I call you',
 'whats your name?',
 'Could you help me?',
 'give me a hand please',
 'Can you help?',
 'What can you do for me?',
 'I need a support',
 'I need a help',
 'support me please',
 'I need to create a new account',
 'how to open a new account',
 'I want to create an account',
 'can you create an account for me',
 'how to open a new account',
 'have a complaint',
 'I want to raise a complaint',
 'there is a complaint about a service']

In [9]:
training_labels

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'about',
 'about',
 'about',
 'name',
 'name',
 'name',
 'help',
 'help',
 'help',
 'help',
 'help',
 'help',
 'help',
 'createaccount',
 'createaccount',
 'createaccount',
 'createaccount',
 'createaccount',
 'complaint',
 'complaint',
 'complaint']

In [10]:
responses

[['Hello', 'Hi', 'Hi there'],
 ['See you later', 'Have a nice day', 'Bye! Come back again'],
 ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"],
 ['I.m Joana, your bot assistant', "I'm Joana, an Artificial Intelligent bot"],
 ['You can call me Joana.', "I'm Joana!", 'Just call me as Joana'],
 ['Tell me how can assist you',
  'Tell me your problem to assist you',
  'Yes Sure, How can I support you'],
 ['You can just easily create a new account from our web site',
  'Just go to our web site and follow the guidelines to create a new account'],
 ['Please provide us your complaint in order to assist you',
  'Please mention your complaint, we will reach you and sorry for any inconvenience caused']]

In [11]:
labels

['greeting',
 'goodbye',
 'thanks',
 'about',
 'name',
 'help',
 'createaccount',
 'complaint']

In [12]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [13]:
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

# Tokenization

In [14]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [15]:
word_index

{'<OOV>': 1,
 'a': 3,
 'about': 51,
 'account': 7,
 'an': 28,
 'anyone': 31,
 'are': 12,
 'bye': 34,
 'call': 43,
 'can': 13,
 'complaint': 17,
 'could': 45,
 'create': 15,
 'do': 48,
 'for': 11,
 'give': 46,
 'goodbye': 37,
 'hand': 47,
 'have': 49,
 'hay': 33,
 'hello': 32,
 'help': 8,
 'helpful': 40,
 'hey': 30,
 'hi': 29,
 'how': 25,
 'i': 4,
 'is': 10,
 'later': 36,
 'me': 5,
 'name': 22,
 'need': 14,
 'new': 16,
 'open': 26,
 'please': 23,
 'raise': 50,
 'see': 35,
 'service': 52,
 'should': 42,
 'support': 24,
 'thank': 38,
 'thanks': 19,
 "that's": 39,
 'the': 41,
 'there': 18,
 'to': 6,
 'want': 27,
 'what': 9,
 'whats': 44,
 'who': 20,
 'you': 2,
 'your': 21}

In [16]:
sequences

[[29],
 [30],
 [10, 31, 18],
 [32],
 [33],
 [34],
 [35, 2, 36],
 [37],
 [19],
 [38, 2],
 [39, 40],
 [19, 11, 41, 8],
 [20, 12, 2],
 [9, 12, 2],
 [20, 2, 12],
 [9, 10, 21, 22],
 [9, 42, 4, 43, 2],
 [44, 21, 22],
 [45, 2, 8, 5],
 [46, 5, 3, 47, 23],
 [13, 2, 8],
 [9, 13, 2, 48, 11, 5],
 [4, 14, 3, 24],
 [4, 14, 3, 8],
 [24, 5, 23],
 [4, 14, 6, 15, 3, 16, 7],
 [25, 6, 26, 3, 16, 7],
 [4, 27, 6, 15, 28, 7],
 [13, 2, 15, 28, 7, 11, 5],
 [25, 6, 26, 3, 16, 7],
 [49, 3, 17],
 [4, 27, 6, 50, 3, 17],
 [18, 10, 3, 17, 51, 3, 52]]

In [17]:
padded_sequences

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 29],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 30],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 10, 31, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 32],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 33],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 34],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 35,  2, 36],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 37],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 19],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0, 38,  2],
       [ 0,  0,  0,  0,  0,  0

# Training a Neural Network

In [18]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
2/2 [==============================] - 1s 8ms/step - loss: 2.0781 - accuracy: 0.2143
Epoch 2/500
2/2 [====

# Saving The Neural Network

In [19]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


# **Deploy a Chatbot with Python and our Trained Machine Learning Model**

In [21]:
!pip install colorama

In [23]:

import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: Is anyone there?
ChatBot: Hi there
User: Who are you?
ChatBot: I'm Joana, an Artificial Intelligent bot
User: what should I call you
ChatBot: Hello
User: I need a help
ChatBot: Tell me how can assist you
User: I want to raise a complaint
ChatBot: Please mention your complaint, we will reach you and sorry for any inconvenience caused
User: Thanks
ChatBot: Hi
User: Thanks for the help
ChatBot: Tell me your problem to assist you
User: Goodbye
ChatBot: Hi
User: quit


In [ ]:
data = {"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "createaccount",
    "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
    "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    }
]
}

with open('intents.json', 'w') as outfile:
    json.dump(data, outfile)